In [2]:
import pandas as pd
import sqlite3
from sqlite3 import connect
from sqlalchemy import create_engine
from nltk.corpus import stopwords
from textblob import TextBlob
from datetime import date, datetime
from pytz import timezone

In [17]:
df = pd.read_csv('data.csv')

In [265]:
con = sqlite3.connect('/Users/sean/labs/Capstone/TwitterStockAnalysis/sentiment.db')

In [275]:
#df = pd.read_sql_query("SELECT * FROM spark_cleaned", con)
df = pd.read_sql_query("SELECT * FROM spark_cleaned WHERE date_time LIKE '2021-09-04%'", con)
df = df.rename(columns = {'date_time': 'date_time_est'})

In [18]:
df.head()

,index,screen_name,date_time_est,text,retweeted,lang,polarity,subjectivity,sentiment,neg,neu,pos,compound
0,0,Zed18978371,2021-09-04 08:48:31.000000,yup im crazy cryptotesla guy,0.0,en,-0.600000,0.900000,negative,0.375,0.625,0.000,-0.3400
1,1,TeslaOwls,2021-09-04 09:28:16.000000,dont believe ever claimed tesla great build qu...,0.0,en,0.427273,0.680909,positive,0.175,0.585,0.240,0.2960
2,2,vin_wiesel,2021-09-04 09:28:11.000000,media tesla crashes building autopilot,0.0,en,0.000000,0.000000,neutral,0.000,1.000,0.000,0.0000
3,3,Jeeeeee66023780,2021-09-04 09:28:09.000000,many followers,0.0,en,0.500000,0.500000,neutral,0.000,1.000,0.000,0.0000
4,4,developrwannab,2021-09-04 09:28:53.000000,great thanks got add tesla bot thats kind deve...,0.0,en,0.525000,0.712500,positive,0.000,0.423,0.577,0.9468


In [277]:
len(df)

70

In [278]:
print(df.iloc[0, 2])
type(df.iloc[0,2])

2021-09-04 12:48:31.000000


str

In [279]:
eastern = timezone('US/Eastern')
utc = timezone('UTC')

# "%d/%m/%Y %H:%M:%S"
for index, row in df['date_time_est'].iteritems():
    
    created_at = datetime.strptime(row[:-7], '%Y-%m-%d %H:%M:%S')
    utc_created_at = utc.localize(created_at)
    #print(utc_created_at)
    row = utc_created_at.astimezone(eastern)
    df.loc[index, 'date_time_est'] = str(row)
    #print(est_created_at)
    #print(type(row))

In [280]:
df.head(5)

,index,screen_name,date_time_est,text,retweeted,lang
0,0,Zed18978371,2021-09-04 08:48:31-04:00,@NorristownN @RolandStautz Yup - I’m the crazy...,0.0,en
1,1,TeslaOwls,2021-09-04 09:28:16-04:00,@EconomicManBlog @pauluaq1 @jrtolver16 @Tesla ...,0.0,en
2,2,vin_wiesel,2021-09-04 09:28:11-04:00,@nuc_geek Media: “TESLA CRASHES INTO BUILDING ...,0.0,en
3,3,Jeeeeee66023780,2021-09-04 09:28:09-04:00,@TeslaAIBot @Tesla Why you have so many follow...,0.0,en
4,4,developrwannab,2021-09-04 09:28:53-04:00,@ukspacebulletin Great thanks. Now I got to ad...,0.0,en


In [19]:
df_drop_index = df.drop(columns = 'index')

In [20]:
df_drop_index.head()

,screen_name,date_time_est,text,retweeted,lang,polarity,subjectivity,sentiment,neg,neu,pos,compound
0,Zed18978371,2021-09-04 08:48:31.000000,yup im crazy cryptotesla guy,0.0,en,-0.600000,0.900000,negative,0.375,0.625,0.000,-0.3400
1,TeslaOwls,2021-09-04 09:28:16.000000,dont believe ever claimed tesla great build qu...,0.0,en,0.427273,0.680909,positive,0.175,0.585,0.240,0.2960
2,vin_wiesel,2021-09-04 09:28:11.000000,media tesla crashes building autopilot,0.0,en,0.000000,0.000000,neutral,0.000,1.000,0.000,0.0000
3,Jeeeeee66023780,2021-09-04 09:28:09.000000,many followers,0.0,en,0.500000,0.500000,neutral,0.000,1.000,0.000,0.0000
4,developrwannab,2021-09-04 09:28:53.000000,great thanks got add tesla bot thats kind deve...,0.0,en,0.525000,0.712500,positive,0.000,0.423,0.577,0.9468


In [21]:
len(df_drop_index)

17740

In [22]:
# remove duplicates 
df_no_dupes = df_drop_index.drop_duplicates()
df_no_dupes
len(df_no_dupes)

17740

In [285]:
# use regex to remove @usernames

df_no_dupes['text'] = df_no_dupes['text'].str.replace('@[^\s]+','')
#re.sub('@[^\s]+','',Tweet)

<ipython-input-285-59960d4ecb67>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df_no_dupes['text'] = df_no_dupes['text'].str.replace('@[^\s]+','')
<ipython-input-285-59960d4ecb67>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_dupes['text'] = df_no_dupes['text'].str.replace('@[^\s]+','')


In [286]:
df_no_dupes

,screen_name,date_time_est,text,retweeted,lang
0,Zed18978371,2021-09-04 08:48:31-04:00,Yup - I’m the crazy crypto/Tesla guy..,0.0,en
1,TeslaOwls,2021-09-04 09:28:16-04:00,I don't believe I ever claimed Tesla had ...,0.0,en
2,vin_wiesel,2021-09-04 09:28:11-04:00,Media: “TESLA CRASHES INTO BUILDING WHILE ON ...,0.0,en
3,Jeeeeee66023780,2021-09-04 09:28:09-04:00,Why you have so many followers ?,0.0,en
4,developrwannab,2021-09-04 09:28:53-04:00,Great thanks. Now I got to add Tesla bot 🤔 So...,0.0,en
5,luke_deluxe,2021-09-04 09:28:44-04:00,Fine. Tesla should.,0.0,en
8,lordhyperkill,2021-09-04 10:52:30-04:00,That’s a real thing? Whenever I have heard a...,0.0,en
9,barryagall,2021-09-04 10:52:29-04:00,White interior.,0.0,en
40,Zed18978371,2021-09-04 08:48:31-04:00,Yup - I’m the crazy crypto/Tesla guy..,NaN,en
41,TeslaOwls,2021-09-04 09:28:16-04:00,I don't believe I ever claimed Tesla had ...,NaN,en


In [287]:
# use regex to remove punctuation and emojis 
df_no_dupes["text"] = df_no_dupes["text"].str.replace('[^\w\s]','')

<ipython-input-287-0fde4d1c2c14>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_no_dupes["text"] = df_no_dupes["text"].str.replace('[^\w\s]','')
<ipython-input-287-0fde4d1c2c14>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_dupes["text"] = df_no_dupes["text"].str.replace('[^\w\s]','')


In [23]:
for index, row in df_no_dupes['date_time_est'].iteritems():
    #print(row[:10])
    # time is currently utc
    df_no_dupes.loc[index, 'time'] = row[11:19]
    df_no_dupes.loc[index, 'new_date'] = row[:10]
df_no_dupes.head()

,screen_name,date_time_est,text,retweeted,lang,polarity,subjectivity,sentiment,neg,neu,pos,compound,time,new_date
0,Zed18978371,2021-09-04 08:48:31.000000,yup im crazy cryptotesla guy,0.0,en,-0.600000,0.900000,negative,0.375,0.625,0.000,-0.3400,08:48:31,2021-09-04
1,TeslaOwls,2021-09-04 09:28:16.000000,dont believe ever claimed tesla great build qu...,0.0,en,0.427273,0.680909,positive,0.175,0.585,0.240,0.2960,09:28:16,2021-09-04
2,vin_wiesel,2021-09-04 09:28:11.000000,media tesla crashes building autopilot,0.0,en,0.000000,0.000000,neutral,0.000,1.000,0.000,0.0000,09:28:11,2021-09-04
3,Jeeeeee66023780,2021-09-04 09:28:09.000000,many followers,0.0,en,0.500000,0.500000,neutral,0.000,1.000,0.000,0.0000,09:28:09,2021-09-04
4,developrwannab,2021-09-04 09:28:53.000000,great thanks got add tesla bot thats kind deve...,0.0,en,0.525000,0.712500,positive,0.000,0.423,0.577,0.9468,09:28:53,2021-09-04


In [24]:
df_no_dupes = df_no_dupes.drop(columns = 'date_time_est')

In [25]:
df_no_dupes.head()

,screen_name,text,retweeted,lang,polarity,subjectivity,sentiment,neg,neu,pos,compound,time,new_date
0,Zed18978371,yup im crazy cryptotesla guy,0.0,en,-0.600000,0.900000,negative,0.375,0.625,0.000,-0.3400,08:48:31,2021-09-04
1,TeslaOwls,dont believe ever claimed tesla great build qu...,0.0,en,0.427273,0.680909,positive,0.175,0.585,0.240,0.2960,09:28:16,2021-09-04
2,vin_wiesel,media tesla crashes building autopilot,0.0,en,0.000000,0.000000,neutral,0.000,1.000,0.000,0.0000,09:28:11,2021-09-04
3,Jeeeeee66023780,many followers,0.0,en,0.500000,0.500000,neutral,0.000,1.000,0.000,0.0000,09:28:09,2021-09-04
4,developrwannab,great thanks got add tesla bot thats kind deve...,0.0,en,0.525000,0.712500,positive,0.000,0.423,0.577,0.9468,09:28:53,2021-09-04


In [13]:
# select only certain time periods, keeping note gmt to est 


In [299]:
# make all text lowercase

df_no_dupes['text'] = df_no_dupes['text'].apply(lambda x: x.lower())

In [300]:
# remove stop words 
stop = stopwords.words('english')

df_no_dupes['text'] = df_no_dupes['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [301]:
# stop
# list of stop words

In [26]:
# clean up indexes
df = df_no_dupes
df = df.reset_index(drop=True)
df

,screen_name,text,retweeted,lang,polarity,subjectivity,sentiment,neg,neu,pos,compound,time,new_date
0,Zed18978371,yup im crazy cryptotesla guy,0.0,en,-0.600000,0.900000,negative,0.375,0.625,0.000,-0.3400,08:48:31,2021-09-04
1,TeslaOwls,dont believe ever claimed tesla great build qu...,0.0,en,0.427273,0.680909,positive,0.175,0.585,0.240,0.2960,09:28:16,2021-09-04
2,vin_wiesel,media tesla crashes building autopilot,0.0,en,0.000000,0.000000,neutral,0.000,1.000,0.000,0.0000,09:28:11,2021-09-04
3,Jeeeeee66023780,many followers,0.0,en,0.500000,0.500000,neutral,0.000,1.000,0.000,0.0000,09:28:09,2021-09-04
4,developrwannab,great thanks got add tesla bot thats kind deve...,0.0,en,0.525000,0.712500,positive,0.000,0.423,0.577,0.9468,09:28:53,2021-09-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17735,NobleChristos,unique nft signed bid win genuine signed tesla...,NaN,en,0.393750,0.475000,positive,0.000,0.877,0.123,0.5859,23:39:32,2021-09-04
17736,manyfacedgoddd,tesla going 0,NaN,en,0.000000,0.000000,neutral,0.000,1.000,0.000,0.0000,23:39:19,2021-09-04
17737,bruceba12492082,httpstcokrzbdngqrk lt trusted channel crypto s...,NaN,en,0.250000,0.250000,positive,0.000,0.744,0.256,0.4767,23:39:01,2021-09-04
17738,VtopiaModerna,tesla skies french startup wants take zeroemis...,NaN,en,0.100000,0.050000,neutral,0.000,1.000,0.000,0.0000,23:38:46,2021-09-04


In [303]:
# sentiment analysis packages

from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [304]:
# nlp 

df[['polarity', 'subjectivity']] = df['text'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))

In [305]:
df.head()

,screen_name,text,retweeted,lang,time,new_date,polarity,subjectivity
0,Zed18978371,yup im crazy cryptotesla guy,0.0,en,8:48:31,2021-09-04,-0.600000,0.900000
1,TeslaOwls,dont believe ever claimed tesla great build qu...,0.0,en,9:28:16,2021-09-04,0.427273,0.680909
2,vin_wiesel,media tesla crashes building autopilot,0.0,en,9:28:11,2021-09-04,0.000000,0.000000
3,Jeeeeee66023780,many followers,0.0,en,9:28:09,2021-09-04,0.500000,0.500000
4,developrwannab,great thanks got add tesla bot thats kind deve...,0.0,en,9:28:53,2021-09-04,0.525000,0.712500


In [306]:
# NLP for sentiment, neg, neu, pos, compound

for index, row in df['text'].iteritems():
    score = SentimentIntensityAnalyzer().polarity_scores(row)
    #print(type(score))
    #print(score)
    neg = score['neg']
    neu = score['neu']
    pos = score['pos']
    comp = score['compound']
    if neg > pos:
        # loc looks at curent row and if neg > pos, sentiment is negative
        df.loc[index, 'sentiment'] = 'negative'
    elif pos > neg:
        df.loc[index, 'sentiment'] = 'positive'
    else:
        df.loc[index, 'sentiment'] = 'neutral'
        
    df.loc[index, 'neg'] = neg
    df.loc[index, 'neu'] = neu
    df.loc[index, 'pos'] = pos
    df.loc[index, 'compound'] = comp
    

In [307]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   screen_name   16 non-null     object 
 1   text          16 non-null     object 
 2   retweeted     8 non-null      float64
 3   lang          16 non-null     object 
 4   time          16 non-null     object 
 5   new_date      16 non-null     object 
 6   polarity      16 non-null     float64
 7   subjectivity  16 non-null     float64
 8   sentiment     16 non-null     object 
 9   neg           16 non-null     float64
 10  neu           16 non-null     float64
 11  pos           16 non-null     float64
 12  compound      16 non-null     float64
dtypes: float64(7), object(6)
memory usage: 1.8+ KB


In [15]:
# data lake > spark cleaned > NLP cleaned > visualization 
df.head(10)

,screen_name,text,retweeted,lang,polarity,subjectivity,sentiment,neg,neu,pos,compound,time,new_date
0,Zed18978371,yup im crazy cryptotesla guy,0.0,en,-0.600000,0.900000,negative,0.375,0.625,0.000,-0.3400,8:48:31,2021-09-04
1,TeslaOwls,dont believe ever claimed tesla great build qu...,0.0,en,0.427273,0.680909,positive,0.175,0.585,0.240,0.2960,9:28:16,2021-09-04
2,vin_wiesel,media tesla crashes building autopilot,0.0,en,0.000000,0.000000,neutral,0.000,1.000,0.000,0.0000,9:28:11,2021-09-04
3,Jeeeeee66023780,many followers,0.0,en,0.500000,0.500000,neutral,0.000,1.000,0.000,0.0000,9:28:09,2021-09-04
4,developrwannab,great thanks got add tesla bot thats kind deve...,0.0,en,0.525000,0.712500,positive,0.000,0.423,0.577,0.9468,9:28:53,2021-09-04
5,luke_deluxe,fine tesla,0.0,en,0.416667,0.500000,positive,0.000,0.357,0.643,0.2023,9:28:44,2021-09-04
6,lordhyperkill,thats real thing whenever heard thought people...,0.0,en,-0.066667,0.233333,positive,0.000,0.929,0.071,0.2263,0:52:30,2021-09-04
7,barryagall,white interior,0.0,en,0.000000,0.000000,neutral,0.000,1.000,0.000,0.0000,0:52:29,2021-09-04
8,future_yas,uncle rich made tesla bot hilarious episode bu...,0.0,en,0.437500,0.875000,positive,0.000,0.717,0.283,0.7430,6:48:38,2021-09-05
9,thatdavemarsh,angle tesla doesnt repair procedures extent ty...,0.0,en,0.500000,0.500000,negative,0.168,0.681,0.152,-0.0772,6:48:35,2021-09-05


In [27]:
df.to_csv('data_graph.csv')